# BibRec: Training Random Forest Model

### RF Features: Country, State, Age, Year-of-Publication, Publisher

In [1]:
from bibrec.server.Utils import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

books, users, ratings = get_normalized_data(books_path='data/BX-Books.csv',
                                            users_path='data/BX-Users.csv',
                                            ratings_path='data/BX-Book-Ratings.csv')

/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:12: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=";", encoding="latin-1")
/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:93: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


TypeError: argument of type 'method' is not iterable

In [ ]:
users

In [ ]:
books = books.drop(["isbn"], axis=1)
books

In [ ]:
ratings = ratings.drop(["isbn"], axis=1)
ratings

In [ ]:
# limit data to top-rated books only
top_ratings = ratings.groupby('isbn13').user_id.count().sort_values(ascending=False)
top_ratings = top_ratings[:100]
top_ratings = top_ratings.reset_index()
# user_id => nr of ratings
top_ratings = top_ratings.drop('user_id', axis=1)
top_ratings

In [ ]:
explicit_ratings = ratings[ratings.book_rating != 0]
top_ratings = top_ratings.merge(explicit_ratings, on='isbn13', how='inner')
top_ratings

In [ ]:
filtered_books = books[books.isbn13.isin(top_ratings.isbn13)]
filtered_books

In [ ]:
filtered_users = users[users.user_id.isin(top_ratings.user_id)]
filtered_users

In [ ]:
encoded_books = hot_encode_books(filtered_books)
encoded_books

In [ ]:
encoded_users = hot_encode_users(filtered_users)
encoded_users

In [ ]:
# RF Features: Country, State, Age, Year-of-Publication, Publisher
df_books = encoded_books.filter(regex="isbn13|normalized_year_of_publication|publisher_", axis=1)
df_books

In [ ]:
df_users = encoded_users.filter(regex="user_id|age|country_|state_", axis=1)
df_users

In [ ]:
# df = df_ratings.filter(regex="isbn13|user_id|normalized_rating", axis=1)
df_ratings = top_ratings.filter(regex="isbn13|user_id|book_rating", axis=1)
df_ratings

In [ ]:
df = df_ratings
df = df.merge(df_books)
df = df.merge(df_users)
df

In [ ]:
# Features
# X = df.drop(['user_id', 'isbn13', 'book_rating'], axis=1)
X = df.drop(['user_id', 'book_rating'], axis=1)
# Prediction
Y = df['book_rating']

In [ ]:
X

In [ ]:
Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

In [ ]:
from os.path import exists
import pickle

# Load the model from the file
model_file = "rf5.pkl"
if exists(model_file):
    print("Loading file:", model_file)
    with open(model_file, "rb") as file:
        rfc = pickle.load(file)
else:
    print("Creating new model")
    rfc = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf=0, n_jobs=3, random_state=1)

In [ ]:
rfc.fit(X_train, y_train)

In [ ]:
import pickle

# Save the model to a file
with open(model_file, "wb") as file:
    pickle.dump(rfc, file)

In [ ]:
rfc_pred = rfc.predict(X_test)
rfc_pred

In [ ]:
print(classification_report(y_test, rfc_pred))
print(accuracy_score(y_test, rfc_pred) * 100)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

classifier = rfc

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix, without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        classifier,
        X_test,
        y_test,
        # display_labels=class_names,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [ ]:
rfc.score(X_train, y_train)

In [ ]:
feature_importance = pd.DataFrame({'Feature_names': X.columns, 'Importances': rfc.feature_importances_})
feature_importance = feature_importance.sort_values(by='Importances', ascending=False)
feature_importance

In [ ]:
plt.bar(feature_importance['Feature_names'], feature_importance['Importances'])
plt.show()

In [ ]:
country_importance = feature_importance.loc[lambda x: x['Feature_names'].str.contains("^country", regex=True)]
country_importance

In [ ]:
state_importance = feature_importance.loc[lambda x: x['Feature_names'].str.contains("^state", regex=True)]
state_importance

In [ ]:
publisher_importance = feature_importance.loc[lambda x: x['Feature_names'].str.contains("^publisher", regex=True)]
publisher_importance

In [ ]:
age_importance = feature_importance.loc[lambda x: x['Feature_names'].str.contains("age", regex=True)]["Importances"]
print("age:", age_importance)
print("country_importance:", country_importance["Importances"].sum())
print("state_importance:", state_importance["Importances"].sum())
print("publisher_importance:", publisher_importance["Importances"].sum())

In [ ]:
importances = feature_importance.loc[lambda x: x['Feature_names'].str.contains("age|normalized_|isbn13", regex=True)]
# pd.append([feature_importance, country_importance])
pd.DataFrame(["country", country_importance["Importances"].sum()], columns=["Feature_names", "Importances"])

pd.DataFrame([1,2], columns=["Feature_names", "Importances"])
# importances = feature_importance.append()
# importances.append(["country", country_importance["Importances"].sum()])
# pd.concat([feature_importance, state_importance])
# pd.concat([feature_importance, publisher_importance])
# plt.bar(feature_importance['Feature_names'], feature_importance['Importances'])
# plt.show()